<a href="https://colab.research.google.com/github/shaheer-2000/Multiple-Pattern-Matching-Using-Aho-Corasick/blob/main/Multiple_Pattern_Matching_Using_Aho_Corasick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Pattern Matching Using CUDA (Aho-Corasick)

# Environment Setup

## Initialization

Remove old installations of cuda and update packages

In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


## Install CUDA 9.2 Toolkit

In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-01-09 10:10:06--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-01-09 10:10:06--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?FDKbqC2Z17GX7INYWdYl9PhpLxrNfb_ta7cLB-sJcuZrXlOo7EJw_u4aOxy14JVYXfZ-uq-zippHdtHp25RtuZTzFj3pOrQrjY_1Nz5BOKQQZ

## Check NVCC Version

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


## Install Package To Execute CUDA (.cu) Code Directly In Notebook Cells

In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-nb3mgu_c
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-nb3mgu_c
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=3d36859226c00d8439337cca7d12ebebcfa9b0311b297d52179b96965b3e6d73
  Stored in directory: /tmp/pip-ephem-wheel-cache-obwbychy/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


## Load The NVCC_PLUGIN

In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# Environment Setup (MPI)

## Initialization

In [50]:
!sudo apt install libopenmpi-dev
!pip install mpi4py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenmpi-dev is already the newest version (2.1.1-8).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
     |████████████████████████████████| 2.5 MB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185267 sha256=72a0e95328b545b43ff1c37796a43f8a0401c8d6da242816650cb81b8af39a15
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py


# Code Execution

## Data Generation

In [38]:
%%cu

#include <bits/stdc++.h>
#include <ctime>

#define FILE_SIZE_MB 1

using namespace std;

int main() {
	int a[100];
	unsigned int * b = (unsigned int*)a;

	string P = "ATGC";
	double n = FILE_SIZE_MB;
	
	ofstream output_file("data.txt");

	srand(time(NULL));

	for(long long int i=0; i< (int) 1024*1024*n; i++) {
      output_file << P[rand() % 4];
  }
}

## Get Active Device

In [87]:
%%cu

#include <stdio.h>

int main() {

  int nDevices;
  cudaGetDeviceCount(&nDevices);
  
  printf("Number of devices: %d\n\n\n", nDevices);
  
  for (int i = 0; i < nDevices; i++) {
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, i);
    printf("---------------------------------\n");
    printf("Device Number: %d\n", i);
    printf("\n");
    printf("Device name: %s\n", prop.name);
    printf("Memory Clock Rate (MHz): %d\n",
           prop.memoryClockRate/1024);
    printf("Memory Bus Width (bits): %d\n",
           prop.memoryBusWidth);
    printf("Peak Memory Bandwidth (GB/s): %.1f\n",
           2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
    printf("Total global memory (Gbytes) %.1f\n",(float)(prop.totalGlobalMem)/1024.0/1024.0/1024.0);
    printf("Shared memory per block (Kbytes) %.1f\n",(float)(prop.sharedMemPerBlock)/1024.0);
    printf("Max Threads per Blocks: %d\n", prop.maxThreadsPerBlock);
    printf("Multiprocessor Count (SMs): %d\n", prop.multiProcessorCount);
    printf("minor-major: %d-%d\n", prop.minor, prop.major);
    printf("Warp-size: %d\n", prop.warpSize);
    printf("Concurrent kernels: %s\n", prop.concurrentKernels ? "yes" : "no");
    printf("Concurrent computation/communication: %s\n\n",prop.deviceOverlap ? "yes" : "no");
    printf("---------------------------------\n\n");
  }
}

Number of devices: 1


---------------------------------
Device Number: 0

Device name: Tesla K80
Memory Clock Rate (MHz): 2446
Memory Bus Width (bits): 384
Peak Memory Bandwidth (GB/s): 240.5
Total global memory (Gbytes) 11.2
Shared memory per block (Kbytes) 48.0
Max Threads per Blocks: 1024
Multiprocessor Count (SMs): 13
minor-major: 7-3
Warp-size: 32
Concurrent kernels: yes
Concurrent computation/communication: yes

---------------------------------




## Write The Header File To Our Notebook

In [34]:
%%writefile project-header.h

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <cuda.h>
#include <fstream>
#include <iostream>
#include <sstream>
#include <queue>
using namespace std;

#define M 3
#define D 6

const int MAXS = M*D + 1;
 
const int MAXC = 26;
 
unsigned int out[MAXS];
 
unsigned int f[MAXS]; 

int g[MAXS][MAXC];

static void checkCUDAError(const char *msg) {

	cudaError_t err = cudaGetLastError();

	if (cudaSuccess != err) {
		fprintf(stderr, "Cuda error: %s: %s.\n", msg, cudaGetErrorString(err));
		exit(EXIT_FAILURE);
	}
}

// This will output the proper CUDA error strings in the event that a CUDA host call returns an error
#define checkCudaErrors(err)           __checkCudaErrors (err, __FILE__, __LINE__)

inline static void __checkCudaErrors(cudaError err, const char *file,
		const int line) {

	if (cudaSuccess != err) {
		fprintf(stderr, "%s(%i) : CUDA Runtime API error %d: %s.\n", file, line,
				(int) err, cudaGetErrorString(err));
		exit(-1);
	}
}


int buildMatchingMachine(string arr[], int k)
{
    // Initialize all values in output function as 0.
    memset(out, 0, sizeof out);
 
    // Initialize all values in goto function as -1.
    memset(g, -1, sizeof g);
 
    // Initially, we just have the 0 state
    int states = 1;
 
    // Construct values for goto function, i.e., fill g[][]
    // This is same as building a Trie for arr[]
    for (int i = 0; i < k; ++i)
    {
        const string &word = arr[i];
        int currentState = 0;
 
        // Insert all characters of current word in arr[]
        for (int j = 0; j < word.size(); ++j)
        {
            int ch = word[j] - 'A';
 
            // Allocate a new node (create a new state) if a
            // node for ch doesn't exist.
            if (g[currentState][ch] == -1)
                g[currentState][ch] = states++;
 
            currentState = g[currentState][ch];
        }
 
        // Add current word in output function
        out[currentState] += 1;
    }
 
    // For all characters which don't have an edge from
    // root (or state 0) in Trie, add a goto edge to state
    // 0 itself
    for (int ch = 0; ch < MAXC; ++ch)
        if (g[0][ch] == -1)
            g[0][ch] = 0;
 
    // Now, let's build the failure function
 
    // Initialize values in fail function
    memset(f, -1, sizeof f);
 
    // Failure function is computed in breadth first order
    // using a queue
    queue<int> q;
 
     // Iterate over every possible input
    for (int ch = 0; ch < MAXC; ++ch)
    {
        // All nodes of depth 1 have failure function value
        // as 0. For example, in above diagram we move to 0
        // from states 1 and 3.
        if (g[0][ch] != 0)
        {
            f[g[0][ch]] = 0;
            q.push(g[0][ch]);
        }
    }
 
    // Now queue has states 1 and 3
    while (q.size())
    {
        // Remove the front state from queue
        int state = q.front();
        q.pop();
 
        // For the removed state, find failure function for
        // all those characters for which goto function is
        // not defined.
        for (int ch = 0; ch <= MAXC; ++ch)
        {
            // If goto function is defined for character 'ch'
            // and 'state'
            if (g[state][ch] != -1)
            {
                // Find failure state of removed state
                int failure = f[state];
 
                // Find the deepest node labeled by proper
                // suffix of string from root to current
                // state.
                while (g[failure][ch] == -1)
                      failure = f[failure];
 
                failure = g[failure][ch];
                f[g[state][ch]] = failure;
 
                // Merge output values
                out[g[state][ch]] += out[failure];
 
                // Insert the next level node (of Trie) in Queue
                q.push(g[state][ch]);
            }
        }
    }
 
    return states;
}

Writing project-header.h


## Execute CUDA Code

In [48]:
%%cu

#include "/content/project-header.h"

__global__ void global_memory_kernel(int *d_go_to_state, unsigned int *d_failure_state, unsigned int *d_output_state, unsigned char *d_text, unsigned int *d_out, size_t pitch, int m, int n, int p_size, int alphabet, int num_blocks ) {
	
	int idx = blockIdx.x * blockDim.x + threadIdx.x;
	int effective_pitch = pitch / sizeof ( int );
	
	int chars_per_block = n / num_blocks;
	
	int start_block = blockIdx.x * chars_per_block;
	int stop_block = start_block + chars_per_block;
	
	int chars_per_thread = ( stop_block - start_block ) / blockDim.x;
	
	int start_thread = start_block + chars_per_thread * threadIdx.x;
	int stop_thread;
	if( blockIdx.x == num_blocks -1 && threadIdx.x==blockDim.x-1)
		stop_thread = n - 1;	
	else stop_thread = start_thread + chars_per_thread + m-1;

	int r = 0, s;
	
	int column;
	
	for ( column = start_thread; ( column < stop_thread && column < n ); column++ ) {

		while ( ( s = d_go_to_state[r * effective_pitch + (d_text[column]-(unsigned char)'A')] ) == -1 )
			r = d_failure_state[r];
		r = s;
			
		d_out[idx] += d_output_state[r];
	}
}




void global_memory_wrapper_func ( int m, unsigned char *text, int n, int p_size, int alphabet, int *go_to_state, unsigned int *failure_state, unsigned int *output_state ) {

	//Pointer for device memory
	int *d_go_to_state;
	unsigned int *d_failure_state, *d_output_state, *d_out;
	
	unsigned char *d_text;

	size_t pitch;
	
	int num_blocks = 8, num_threads_per_block = 1024;

	dim3 dimGrid ( num_blocks );
	dim3 dimBlock ( num_threads_per_block );
	
	if ( n < num_blocks * num_threads_per_block * m ) {
		printf("The text size is too small\n");
		exit(1);
	}
	
	//Allocate host memory for results array
	unsigned int *h_out = ( unsigned int * ) malloc ( num_blocks * num_threads_per_block * sizeof ( unsigned int ) );
	memset ( h_out, 0, num_blocks * num_threads_per_block * sizeof ( unsigned int ) );

	//Allocate 1D device memory
	checkCudaErrors ( cudaMalloc ( ( void** ) &d_text, n * sizeof ( unsigned char ) ) );
	checkCudaErrors ( cudaMalloc ( ( void** ) &d_failure_state, ( m * p_size + 1 ) * sizeof ( unsigned int ) ) );
	checkCudaErrors ( cudaMalloc ( ( void** ) &d_output_state, ( m * p_size + 1 ) * sizeof ( unsigned int ) ) );
	checkCudaErrors ( cudaMalloc ( ( void** ) &d_out, num_blocks * num_threads_per_block * sizeof ( unsigned int ) ) );
	
	//Allocate 2D device memory
	checkCudaErrors ( cudaMallocPitch ( &d_go_to_state, &pitch, alphabet * sizeof ( int ), ( m * p_size + 1 ) ) );
	
	//Copy 1D host memory to device
	checkCudaErrors ( cudaMemcpy ( d_text, text, n * sizeof ( unsigned char ), cudaMemcpyHostToDevice ) );
	checkCudaErrors ( cudaMemcpy ( d_failure_state, failure_state, ( m * p_size + 1 ) * sizeof ( unsigned int ), cudaMemcpyHostToDevice ) );
	checkCudaErrors ( cudaMemcpy ( d_output_state, output_state, ( m * p_size + 1 ) * sizeof ( unsigned int ), cudaMemcpyHostToDevice ) );
	checkCudaErrors ( cudaMemcpy ( d_out, h_out, num_blocks * num_threads_per_block * sizeof ( unsigned int ), cudaMemcpyHostToDevice ) );

	//Copy 2D host memory to device
	checkCudaErrors ( cudaMemcpy2D ( d_go_to_state, pitch, go_to_state, alphabet * sizeof ( int ), alphabet * sizeof ( int ), ( m * p_size + 1 ), cudaMemcpyHostToDevice ) );
	
	//Create timer
	cudaEvent_t start, stop;

	float time;
	
	//Create the timer events
	cudaEventCreate ( &start );
	cudaEventCreate ( &stop );
	
	//Start the event clock	
	cudaEventRecord ( start, 0 );
	
	
	
	//Executing kernel in the device
	global_memory_kernel<<<dimGrid, dimBlock>>>( d_go_to_state, d_failure_state, d_output_state, d_text, d_out, pitch, m, n, p_size, alphabet, num_blocks );
	checkCUDAError("kernel invocation");
	

	cudaEventRecord ( stop, 0 );

	cudaEventSynchronize ( stop );
	
	cudaEventElapsedTime ( &time, start, stop );
	
	cudaEventDestroy ( start );
	cudaEventDestroy ( stop );
	
	//Get back the results from the device
	cudaMemcpy ( h_out, d_out, num_blocks * num_threads_per_block * sizeof ( unsigned int ), cudaMemcpyDeviceToHost );
	
  	//Look at the results
  	int i, matches = 0;
  	for ( i = 0; i < num_blocks * num_threads_per_block; i++ )
  		matches += h_out[i];
  		
	printf ("Global Memory Kernel 1 matches \t%i\t time \t%fms\n", matches, time);
		
	//Free host and device memory
	free ( h_out );
	
	cudaFree ( d_text );
	cudaFree ( d_go_to_state );
	cudaFree ( d_failure_state );
	cudaFree ( d_output_state );
	cudaFree ( d_out );
}




int main(){

 	  string patterns[]={"ATC","GTG","GTC","ATG","CAA","ATT"};
    // string patterns[]={"ATC"};
    int k = sizeof( patterns )/sizeof( patterns[0] );
    string text;

    ifstream t( "data.txt" );
    stringstream buffer;
    buffer << t.rdbuf();    
    text = buffer.str();
    unsigned char *charText = ( unsigned char* )text.c_str();
    
    buildMatchingMachine(patterns, k);

    int *go_to_table = (int*)malloc( sizeof(int)*MAXC*MAXS );
    
    for(int i=0;i<MAXS;i++)
        for(int j=0;j<MAXC;j++)
            go_to_table[i*MAXC+j] = g[i][j];
	
	global_memory_wrapper_func( M, charText, text.size(), D, 26, go_to_table, f, out );
	
	return 0;
}


Global Memory Kernel 1 matches 	98550	 time 	1.035072ms



# Code Execution (Serial)

## Execute Serial Code

In [49]:
%%cu

// C++ program for implementation of Aho Corasick algorithm
// for string matching
using namespace std;
#include <bits/stdc++.h>
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include<fstream>
#include<iostream>
#include <sstream>
#include<queue>
#include <ctime> 
// Max number of states in the matching machine.
// Should be equal to the sum of the length of all keywords.
const int MAXS = 19;
 
// Maximum number of characters in input alphabet
const int MAXC = 26;
 
// OUTPUT FUNCTION IS IMPLEMENTED USING out[]
// Bit i in this mask is one if the word with index i
// appears when the machine enters this state.
int out[MAXS];
 
// FAILURE FUNCTION IS IMPLEMENTED USING f[]
int f[MAXS];
 
// GOTO FUNCTION (OR TRIE) IS IMPLEMENTED USING g[][]
int g[MAXS][MAXC];

clock_t start,stop;
 
// Builds the string matching machine.
// arr -   array of words. The index of each keyword is important:
//         "out[state] & (1 << i)" is > 0 if we just found word[i]
//         in the text.
// Returns the number of states that the built machine has.
// States are numbered 0 up to the return value - 1, inclusive.
int buildMatchingMachine(string arr[], int k)
{
    // Initialize all values in output function as 0.
    memset(out, 0, sizeof out);
 
    // Initialize all values in goto function as -1.
    memset(g, -1, sizeof g);
 
    // Initially, we just have the 0 state
    int states = 1;
 
    // Construct values for goto function, i.e., fill g[][]
    // This is same as building a Trie for arr[]
    for (int i = 0; i < k; ++i)
    {
        const string &word = arr[i];
        int currentState = 0;
 
        // Insert all characters of current word in arr[]
        for (int j = 0; j < word.size(); ++j)
        {
            int ch = word[j] - 'A';
 
            // Allocate a new node (create a new state) if a
            // node for ch doesn't exist.
            if (g[currentState][ch] == -1)
                g[currentState][ch] = states++;
 
            currentState = g[currentState][ch];
        }
 
        // Add current word in output function
        //out[currentState] |= (1 << i);
        out[currentState] += 1;
    }
 
    // For all characters which don't have an edge from
    // root (or state 0) in Trie, add a goto edge to state
    // 0 itself
    for (int ch = 0; ch < MAXC; ++ch)
        if (g[0][ch] == -1)
            g[0][ch] = 0;
 
    // Now, let's build the failure function
 
    // Initialize values in fail function
    memset(f, -1, sizeof f);
 
    // Failure function is computed in breadth first order
    // using a queue
    queue<int> q;
 
     // Iterate over every possible input
    for (int ch = 0; ch < MAXC; ++ch)
    {
        // All nodes of depth 1 have failure function value
        // as 0. For example, in above diagram we move to 0
        // from states 1 and 3.
        if (g[0][ch] != 0)
        {
            f[g[0][ch]] = 0;
            q.push(g[0][ch]);
        }
    }
 
    // Now queue has states 1 and 3
    while (q.size())
    {
        // Remove the front state from queue
        int state = q.front();
        q.pop();
 
        // For the removed state, find failure function for
        // all those characters for which goto function is
        // not defined.
        for (int ch = 0; ch <= MAXC; ++ch)
        {
            // If goto function is defined for character 'ch'
            // and 'state'
            if (g[state][ch] != -1)
            {
                // Find failure state of removed state
                int failure = f[state];
 
                // Find the deepest node labeled by proper
                // suffix of string from root to current
                // state.
                while (g[failure][ch] == -1)
                      failure = f[failure];
 
                failure = g[failure][ch];
                f[g[state][ch]] = failure;
 
                // Merge output values
                //out[g[state][ch]] |= out[failure];
                out[g[state][ch]] += out[failure];
                // Insert the next level node (of Trie) in Queue
                q.push(g[state][ch]);
            }
        }
    }
 
    return states;
}
 
// Returns the next state the machine will transition to using goto
// and failure functions.
// currentState - The current state of the machine. Must be between
//                0 and the number of states - 1, inclusive.
// nextInput - The next character that enters into the machine.
int findNextState(int currentState, char nextInput)
{
    int answer = currentState;
    int ch = nextInput - 'A';
 
    // If goto is not defined, use failure function
    while (g[answer][ch] == -1)
        answer = f[answer];
 
    return g[answer][ch];
}
 
// This function finds all occurrences of all array words
// in text.
void searchWords(string arr[], int k, string text)
{
    // Preprocess patterns.
    // Build machine with goto, failure and output functions
    buildMatchingMachine(arr, k);
 
 
    // Initialize current state
    int currentState = 0;
    int count=0;
    // Traverse the text through the nuilt machine to find
    // all occurrences of words in arr[]
    start = clock();
    for (int i = 0; i < text.size(); ++i)
    {
        currentState = findNextState(currentState, text[i]);
 
        // If match not found, move to next state
        if (out[currentState] == 0)
             continue;
         /* for (int j = 0; j < k; ++j)
                {
                    if (out[currentState] & (1 << j))
                    {
                       // cout << "Word " << arr[j] << " appears from "
                        //    << i - arr[j].size() + 1 << " to " << i << endl;
                        count++;
                    }
                }*/
                    count+=out[currentState];
    }
        
    stop = clock();
    cout<<" "<<count<<" ";
}
 
// Driver program to test above
int main()
{
    //string arr[] = {"he", "she", "hers", "his"};
    string arr[]={"ATC","GTG","GTC","ATG","CAA","ATT"};
    //string text = "ahishers";
    ifstream t("data.txt");
    stringstream buffer;
    buffer << t.rdbuf();    
    string text = buffer.str();
 
    int k = sizeof(arr)/sizeof(arr[0]);
 

    
    float time;


    searchWords(arr, k, text);


    
    time = (stop-start)*1000/CLOCKS_PER_SEC;
    cout<<" Elapsed Time : "<<time<<"ms\n";
    
    
    
    /*for(int i=0;i<MAXS;i++)
        cout<<out[i]<<" ";
    */
    return 0;
}

 98550  Elapsed Time : 23ms



# Code Execution (OMP)

In [58]:
%%writefile node.h

/*
	Copyright 2010-2011 Kamiar Kanani <kamiar.kanani@gmail.com>

    This file is part of multifast.

    multifast is free software: you can redistribute it and/or modify
    it under the terms of the GNU Lesser General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    multifast is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Lesser General Public License for more details.

    You should have received a copy of the GNU Lesser General Public License
    along with multifast.  If not, see <http://www.gnu.org/licenses/>.

*/

#ifndef _NODE_H_
#define _NODE_H_

#include "config.h"
#include "ac_types.h"

/* Forward Declaration */
struct edge;

typedef struct node
/* The Node of the Automata */
{
	int id; /* Node ID : for debugging purpose */
	short int final; /* 0: no ; 1: yes, it is a final node */
	struct node * failure_node; /* The failure node of this node */
	unsigned short depth; /* depth: distance between this node to the root */

	/* Matched Strings */
	STRING * matched_strings; /* Array of matched strings */
	unsigned short matched_strings_num; /* Number of matched string at this node */
	unsigned short matched_strings_max; /* Max capacity of allocated memory for 'matched_strings' */

	/* Outgoing Edges */
	struct edge * outgoing; /* Array of outgoing edges */
	unsigned short outgoing_degree; /* Number of outgoing edges */
	unsigned short outgoing_max; /* Max capacity of allocated memory for 'outgoing' */
} NODE;

struct edge
/* The Edge of the Node */
{
	ALPHA alpha; /* Edge alpha */
	struct node * next; /* Target of the edge */
};

/* Public Functions */
NODE * node_create            (void);
NODE * node_create_next       (NODE * thiz, ALPHA alpha);
void   node_register_matchstr (NODE * thiz, STRING * str);
void   node_register_outgoing (NODE * thiz, NODE * next, ALPHA alpha);
NODE * node_find_next         (NODE * thiz, ALPHA alpha);
NODE * node_findbs_next       (NODE * thiz, ALPHA alpha);
void   node_release           (NODE * thiz);
void   node_assign_id         (NODE * thiz);
void   node_sort_edges        (NODE * thiz);

#endif

Writing node.h


In [59]:
%%writefile config.h

/*
	Copyright 2010-2011 Kamiar Kanani <kamiar.kanani@gmail.com>
    This file is part of multifast.
    multifast is free software: you can redistribute it and/or modify
    it under the terms of the GNU Lesser General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.
    multifast is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Lesser General Public License for more details.
    You should have received a copy of the GNU Lesser General Public License
    along with multifast.  If not, see <http://www.gnu.org/licenses/>.
*/

#ifndef _CONFIG_H_
#define _CONFIG_H_

/* Define below macro to enable automata display function: ac_automata_dbg_show() */
#define DEBUG_DISPLAY_AC

#endif


Writing config.h


In [61]:
%%writefile ac_types.h

/*
	Copyright 2010-2011 Kamiar Kanani <kamiar.kanani@gmail.com>

    This file is part of multifast.

    multifast is free software: you can redistribute it and/or modify
    it under the terms of the GNU Lesser General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    multifast is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Lesser General Public License for more details.

    You should have received a copy of the GNU Lesser General Public License
    along with multifast.  If not, see <http://www.gnu.org/licenses/>.

*/

#ifndef _AC_TYPES_H_
#define _AC_TYPES_H_

/* ALPHA

   We use fix alpha size in this implementation
   if you are intended to use variable size codings
   you must convert it to a fix one (see iconv library).

   define your alpha type in below.
*/
typedef char ALPHA;

/* String Identifier 

   Every string may have an identifier to distinguish it
   in Aho-Corasik automata. It may be useful while retrieving 
   strings from a database.
   The defined type must be big enough to enumerate all strings.
*/
typedef unsigned long STRINGID;

/* STRING type */
typedef struct
{
	ALPHA * str; /* Array of ALPHAs */
	unsigned int length; /* Length of string */
	STRINGID id; /* String identifier (Optional) */
} STRING;

/* Maximum Pattern Length */
#define AC_PATTRN_MAX_LENGTH 256

/* Any Match is Reported in the following structure */
typedef struct
{
	/* Array of Matched Strings */
	STRING * matched_strings;
	/* Matched Position:
	   determine the last element (alpha) of string(s) */
	long position;
	/* Number of matched string (length of sid array) */
	unsigned int match_num;
} MATCH;

/* MATCH_CALBACK: 
	Callback function type 
	We use a callback function to report a match accurance
	to the caller.
*/
typedef int (*MATCH_CALBACK)(MATCH *, int, int);

/* Error Numbers */
typedef enum
{
	ACERR_NONE = 0,
	ACERR_DUPLICATE_STRING,
	ACERR_LONG_STRING,
	ACERR_ZERO_STRING,
	ACERR_STRING_CLOSED,
} AC_ERROR;

#endif


Writing ac_types.h


In [56]:
%%writefile aho_corasick.h

/*
	Copyright 2010-2011 Kamiar Kanani <kamiar.kanani@gmail.com>

    This file is part of multifast.

    multifast is free software: you can redistribute it and/or modify
    it under the terms of the GNU Lesser General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    multifast is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Lesser General Public License for more details.

    You should have received a copy of the GNU Lesser General Public License
    along with multifast.  If not, see <http://www.gnu.org/licenses/>.

*/

#ifndef _AUTOMATA_H_
#define _AUTOMATA_H_

#include "/content/config.h"
#include "/content/node.h"

typedef struct
{
	/* The root of the Aho-Corasick trie */
	NODE * root;

	/* maintain all node pointers in this automata.
	   it will be used to traverse or release all nodes.
	*/
	NODE ** all_nodes;
	unsigned int all_nodes_num; /* Number of all nodes in the automata */
	unsigned int all_nodes_max; /* Max capacity of allocated memory for *all_nodes */

	MATCH match; /* Any match is writen in here */
	MATCH_CALBACK match_callback; /* Match callback function */

	/* this is a flag for string acceptance check 
	   after ac_automata_locate_failure() it is set to 0 to indicate end of strings
	   it is set to 1 in ac_automata_init()
	*/
	unsigned int accept_strings;

	/* 
	   following members keep automata state for sake of repeated call for
	   ac_automata_search(); it needed when we deal with larg input string.
	   in that case we break input string into smaller chunk and repeatedly
	   call ac_automata_search().
	*/
	NODE * current_node; /* Pointer to current node while searching */
	unsigned long base_position; /* Represents the position of current chunk related to whole input */

	/* Statistic Variables */
	unsigned long total_strings; /* Total Strings in the Automata */

} AC_AUTOMATA;


/* Public Functions */
void     ac_automata_init           (AC_AUTOMATA * thiz, MATCH_CALBACK mc);
AC_ERROR ac_automata_add_string     (AC_AUTOMATA * thiz, STRING * str);
void     ac_automata_locate_failure (AC_AUTOMATA * thiz);
void     ac_automata_search         (AC_AUTOMATA * thiz, STRING * str, int automata_num, int thread_num);
void     ac_automata_reset          (AC_AUTOMATA * thiz);
void     ac_automata_release        (AC_AUTOMATA * thiz);

#ifdef DEBUG_DISPLAY_AC
void     ac_automata_dbg_show       (AC_AUTOMATA * thiz);
#endif


#endif


Overwriting aho_corasick.h


In [54]:
%%writefile patterns.txt
ATC
GTG
GTC
ATG
CAA
ATT

Overwriting patterns.txt


In [83]:
%%writefile ac_omp.c

#include <math.h>
#include <omp.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <unistd.h>

#include "/content/aho_corasick.h"
#include "/content/node.h"
#include "/content/ac_types.h"

#define NO_OF_THREADS 2
#define MAX_PATTERN_SIZE 128

// aho_corasick.c

/*
	Copyright 2010-2011 Kamiar Kanani <kamiar.kanani@gmail.com>

    This file is part of multifast.

    multifast is free software: you can redistribute it and/or modify
    it under the terms of the GNU Lesser General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    multifast is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Lesser General Public License for more details.

    You should have received a copy of the GNU Lesser General Public License
    along with multifast.  If not, see <http://www.gnu.org/licenses/>.

*/

/* Allocation step for automata::all_nodes array */
#define REALLOC_CHUNK_ALLNODES 200


/* Private Functions */
void   ac_automata_register_nodeptr  (AC_AUTOMATA * thiz, NODE * node);
void   ac_automata_set_failure       (AC_AUTOMATA * thiz, NODE * node, ALPHA * alphas);
void   ac_automata_dfs_traverse      (AC_AUTOMATA * thiz, NODE * node, ALPHA * alphas);
void   ac_automata_union_matchstrs   (NODE * node);



/******************************************************************************
FUNCTION: ac_automata_init
PARAMS:
	AC_AUTOMATA * thiz: Pointer to the automata
	MATCH_CALBACK mc: callback function

DESCRIPTION:
	Initialize automata; allocate memories for automata.
******************************************************************************/
void ac_automata_init (AC_AUTOMATA * thiz, MATCH_CALBACK mc)
{
	memset (thiz, 0, sizeof(AC_AUTOMATA));
	thiz->root = node_create ();
	thiz->all_nodes_max = REALLOC_CHUNK_ALLNODES;
	thiz->all_nodes = (NODE **) malloc (thiz->all_nodes_max*sizeof(NODE *));
	thiz->match_callback = mc;
	ac_automata_register_nodeptr (thiz, thiz->root);
	ac_automata_reset (thiz);
	thiz->total_strings = 0;
	thiz->accept_strings = 1;
}


/******************************************************************************
FUNCTION: ac_automata_reset

DESCRIPTION: 
	reset automata state. when you finished with the input string,
	you must reset automata state for new input, otherwise it will not work.
******************************************************************************/
void ac_automata_reset (AC_AUTOMATA * thiz)
{
	thiz->current_node = thiz->root;
	thiz->base_position = 0;
}


/******************************************************************************
FUNCTION: ac_automata_register_nodeptr

DESCRIPTION:
	Add node pointer to the list (all_nodes) for further use.
******************************************************************************/
void ac_automata_register_nodeptr (AC_AUTOMATA * thiz, NODE * node)
{
	if(thiz->all_nodes_num >= thiz->all_nodes_max)
	{
		thiz->all_nodes_max += REALLOC_CHUNK_ALLNODES;
		thiz->all_nodes = realloc (thiz->all_nodes, thiz->all_nodes_max*sizeof(NODE *));
	}

	thiz->all_nodes[thiz->all_nodes_num++] = node;
}


/******************************************************************************
FUNCTION: ac_automata_add_string

RETUERNS:
	ACERR_NONE on success
	ACERR_LONG_STRING when string length is longer than AC_PATTRN_MAX_LENGTH
	ACERR_DUPLICATE_STRING on duplicte strings
	ACERR_ZERO_STRING on zero length string

DESCRIPTION:
	Add string to the automata.
	CAUTION: If the given string be larger than AC_PATTRN_MAX_LENGTH, it will
	be cropped without any warning.
******************************************************************************/
AC_ERROR ac_automata_add_string (AC_AUTOMATA * thiz, STRING * str)
{
	unsigned int i;
	NODE * n = thiz->root;
	NODE * next;
	ALPHA alpha;

	if(!thiz->accept_strings)
		return ACERR_STRING_CLOSED;

	if (!str->length)
		return ACERR_ZERO_STRING;

	if (str->length > AC_PATTRN_MAX_LENGTH)
		return ACERR_LONG_STRING;

	for (i=0; i<str->length; i++)
	{
		alpha = str->str[i];
		if ((next = node_find_next(n, alpha)))
		{
			n = next;
			continue;
		}
		else
		{
			next = node_create_next(n, alpha);
			next->depth = n->depth + 1;
			n = next;
			ac_automata_register_nodeptr(thiz, n);
		}
	}

	if(n->final)
		return ACERR_DUPLICATE_STRING;

	n->final = 1;
	node_register_matchstr(n, str);
	thiz->total_strings++;

	return ACERR_NONE;
}


/******************************************************************************
FUNCTION: ac_automata_release

DESCRIPTION:
	Release all alocated memories to the automata
******************************************************************************/
void ac_automata_release (AC_AUTOMATA * thiz)
{
	unsigned int i;
	NODE * n;

	for (i=0; i < thiz->all_nodes_num; i++)
	{
		n = thiz->all_nodes[i];
		node_release(n);
	}

	free(thiz->all_nodes);
}


/******************************************************************************
FUNCTION: ac_automata_union_matchstrs

DESCRIPTION:
	Accepted string in any node consists of its own strings plus strings of
	its failure node. this function collects all strings in failure node.
******************************************************************************/
void ac_automata_union_matchstrs (NODE * node)
{
	unsigned int i;
	NODE * m = node;

	while ((m = m->failure_node))
	{
		for (i=0; i < m->matched_strings_num; i++)
			node_register_matchstr(node, &(m->matched_strings[i]));

		if (m->final)
			node->final = 1;
	}
}


/******************************************************************************
FUNCTION: ac_automata_set_failure

DESCRIPTION:
	find failure node for the given node.
******************************************************************************/
void ac_automata_set_failure (AC_AUTOMATA * thiz, NODE * node, ALPHA * alphas)
{
	unsigned int i, j;
	NODE * m;

	for (i=1; i < node->depth; i++)
	{
		m = thiz->root;
		for (j=i; j < node->depth && m; j++)
			m = node_find_next (m, alphas[j]);
		if (m)
		{
			node->failure_node = m;
			break;
		}
	}
	if (!node->failure_node)
		node->failure_node = thiz->root;
}


/******************************************************************************
FUNCTION: ac_automata_dfs_traverse

DESCRIPTION:
	Traverse all automata nodes using DFS (Deapth First Search), meanwile
	it set the failure node for every node it passes through.
*****************************************************************************/
void ac_automata_dfs_traverse (AC_AUTOMATA * thiz, NODE * node, ALPHA * alphas)
{
	unsigned int i;
	NODE * next;

	for (i=0; i < node->outgoing_degree; i++)
	{
		alphas[node->depth] = node->outgoing[i].alpha;
		next = node->outgoing[i].next;

		/* At every node look for its failure node */
		ac_automata_set_failure (thiz, next, alphas);

		/* Recursively call itself to traverse all nodes */
		ac_automata_dfs_traverse (thiz, next, alphas);
	}
}


/******************************************************************************
FUNCTION: ac_automata_locate_failure

DESCRIPTION:
	Locate the failure node for all nodes.
******************************************************************************/
void ac_automata_locate_failure (AC_AUTOMATA * thiz)
{
	unsigned int i;
	ALPHA alphas[AC_PATTRN_MAX_LENGTH];
	NODE * node;

	ac_automata_dfs_traverse (thiz, thiz->root, alphas);

	for (i=0; i < thiz->all_nodes_num; i++)
	{
		node = thiz->all_nodes[i];
		ac_automata_union_matchstrs (node);
		node_sort_edges (node);
	}

	thiz->accept_strings = 0; /* do not accept strings any more */
}


/******************************************************************************
FUNCTION: ac_automata_search

DESCRIPTION:
	Search for patterns inside the given input, as it finds a match
	it will call the Callback functions to report it to caller.
******************************************************************************/
void ac_automata_search (AC_AUTOMATA * thiz, STRING * str, int automata_num, int thread_num)
{
	//printf("\nIn thread %d, automata %d\n", automata_num, thread_num);
	unsigned long position;
	NODE * current;
	NODE * next;

	if(thiz->accept_strings)
		/* you must call ac_automata_locate_failure() first */
		return;

	position = 0;
	/* reload status variable(s) */
	current = thiz->current_node;

	/* This is the main search loop. it must be keep 
	   as lightwaight as possible. */
	while (position < str->length)
	{
		if(!(next = node_findbs_next(current, str->str[position])))
		{
			if(current->failure_node /* we are not in root */)
				current = current->failure_node;
			else
				position++;
		}
		else
		{
			current = next;
			position++;
		}

		if(current->final && next)
		/* We check 'next' to find out if we came here after a alpha transition 
		   or due to a fail. in second case we should not report matching,
		   because it was reported in previous node */
		{
			thiz->match.position = position + thiz->base_position;
			thiz->match.match_num = current->matched_strings_num;
			thiz->match.matched_strings = current->matched_strings;
			/* do callback: we found a match */
			if (thiz->match_callback(&thiz->match, automata_num, thread_num))
				break;
		}
	}

	/* save status variables */
	thiz->current_node = current;
	thiz->base_position += position;
}


/******************************************************************************
FUNCTION: ac_automata_dbg_show

DESCRIPTION:
	Print automata to autput in human readable form.
	used for debugging purpose.
******************************************************************************/
#ifdef DEBUG_DISPLAY_AC
void ac_automata_dbg_show (AC_AUTOMATA * thiz)
{
	unsigned int i, j;
	NODE * n;
	struct edge * e;
	STRING sid;

	for (i=0; i<thiz->all_nodes_num; i++)
	{
		if(!i)
			printf("--------------------------------------\n");
		n = thiz->all_nodes[i];
		printf("NODE(%d)/----FAIL---> NODE(%d)\n", n->id, (n->failure_node)?n->failure_node->id:0);
		for (j=0; j<n->outgoing_degree; j++)
		{
			e = &n->outgoing[j];
			printf("         |----(%c)----> NODE(%d)\n", e->alpha, e->next->id);
		}
		printf("ACCEPTED STRING: {");
		for (j=0; j<n->matched_strings_num; j++)
		{
			sid = n->matched_strings[j];
			printf("%ld ", sid.id);
		}
		printf("}\n");
		printf("--------------------------------------\n");
	}
}
#endif


// node.c

/*
	Copyright 2010-2011 Kamiar Kanani <kamiar.kanani@gmail.com>
    This file is part of multifast.
    multifast is free software: you can redistribute it and/or modify
    it under the terms of the GNU Lesser General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.
    multifast is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Lesser General Public License for more details.
    You should have received a copy of the GNU Lesser General Public License
    along with multifast.  If not, see <http://www.gnu.org/licenses/>.
*/

/* reallocation step for node::matched_strings array */
#define REALLOC_CHUNK_MATCHSTR 8

/* reallocation step for node::outgoing array */
#define REALLOC_CHUNK_OUTGOING 8
/* TODO: For different depth of node, number of outgoing edges differs
considerably, It is efficient to use different chunk size for 
different depths */

/* Private Functions */
void   node_init              (NODE * thiz);
int    node_edge_compare      (const void * l, const void * r);
int    node_has_matchstr      (NODE * thiz, STRING * newstr);



/******************************************************************************
FUNCTION: node_create
DESCRIPTION:
	Create the node
******************************************************************************/
struct node * node_create(void)
{
	NODE * thiz;

	thiz = (NODE *) malloc (sizeof(NODE));
	node_init(thiz);
	#ifdef DEBUG_DISPLAY_AC
	node_assign_id(thiz);
	#endif

	return thiz;
}


/******************************************************************************
FUNCTION: node_init
DESCRIPTION:
	Initialize node
******************************************************************************/
void node_init(NODE * thiz)
{
	memset(thiz, 0, sizeof(NODE));

	thiz->outgoing_max = REALLOC_CHUNK_OUTGOING;
	thiz->outgoing = (struct edge *) malloc (thiz->outgoing_max*sizeof(struct edge));

	thiz->matched_strings_max = REALLOC_CHUNK_MATCHSTR;
	thiz->matched_strings = (STRING *) malloc (thiz->matched_strings_max*sizeof(STRING));
}


/******************************************************************************
FUNCTION: node_release
DESCRIPTION:
	Release node
******************************************************************************/
void node_release(NODE * thiz)
{
	free(thiz->matched_strings);
	free(thiz->outgoing);
	free(thiz);
}


/******************************************************************************
FUNCTION: node_find_next
DESCRIPTION: 
	Find out the target node from here for a given Alpha.
	it uses linear search. this function is used in
	preprocessig stage in which edge array is not sorted.
******************************************************************************/
NODE * node_find_next(NODE * thiz, ALPHA alpha)
{
	int i;

	for (i=0; i < thiz->outgoing_degree; i++)
	{
		if(thiz->outgoing[i].alpha == alpha)
			return (thiz->outgoing[i].next);
	}

	return NULL;
}


/******************************************************************************
FUNCTION: node_findbs_next
DESCRIPTION: 
	Find out the target node from here for a given Alpha.
	it uses Binary Search. this function is used after 
	preprocessing stage in which we sort edges.
******************************************************************************/
NODE * node_findbs_next (NODE * thiz, ALPHA alpha)
{
	int min, max, mid;
	ALPHA amid;

	min = 0;
	max = thiz->outgoing_degree - 1;

	while (min <= max)
	{
		mid = (min+max) >> 1;
		amid = thiz->outgoing[mid].alpha;
		if (alpha > amid)
			min = mid + 1;
		else if (alpha < amid)
			max = mid - 1;
		else
			return (thiz->outgoing[mid].next);
	}

	return NULL;
}


/******************************************************************************
FUNCTION: node_has_matchstr
DESCRIPTION:
	Determine if a final node contains an string in its accepted string list
	Return values: 1 = has, 0 = hasn't
******************************************************************************/
int node_has_matchstr (NODE * thiz, STRING * newstr)
{
	int i, j;
	STRING * str;

	for (i=0; i < thiz->matched_strings_num; i++)
	{
		str = &thiz->matched_strings[i];

		if (str->length != newstr->length)
			continue;

		for (j=0; j<str->length; j++)
			if(str->str[j] != newstr->str[j])
				continue;

		if (j == str->length)
			return 1;
	}

	return 0;
}


/******************************************************************************
FUNCTION: node_create_next
DESCRIPTION:
	Create next node for the given alpha
******************************************************************************/
NODE * node_create_next (NODE * thiz, ALPHA alpha)
{
	NODE * next;
	next = node_find_next (thiz, alpha);
	if (next)
	/* The edge already exists */
		return NULL;
	/* Otherwise register new edge */
	next = node_create ();
	node_register_outgoing(thiz, next, alpha);

	return next;
}


/******************************************************************************
FUNCTION: node_register_matchstr
DESCRIPTION:
	Add the string to the list of accepted strings
******************************************************************************/
void node_register_matchstr(NODE * thiz, STRING * str)
{
	/* Check if the new string already exists in the node list */
	if (node_has_matchstr(thiz, str))
		return;

	/* Manage memory */
	if (thiz->matched_strings_num >= thiz->matched_strings_max)
	{
		thiz->matched_strings_max += REALLOC_CHUNK_MATCHSTR;
		thiz->matched_strings = (STRING *) realloc 
			(thiz->matched_strings, thiz->matched_strings_max*sizeof(STRING));
	}

	thiz->matched_strings[thiz->matched_strings_num].str = str->str;
	thiz->matched_strings[thiz->matched_strings_num].length = str->length;
	thiz->matched_strings[thiz->matched_strings_num].id = str->id;
	thiz->matched_strings_num++;
}


/******************************************************************************
FUNCTION: node_register_outgoing
DESCRIPTION:
	Establish an edge between two nodes
******************************************************************************/
void node_register_outgoing (NODE * thiz, NODE * next, ALPHA alpha)
{
	if(thiz->outgoing_degree >= thiz->outgoing_max)
	{
		thiz->outgoing_max += REALLOC_CHUNK_OUTGOING;
		thiz->outgoing = (struct edge *) realloc 
			(thiz->outgoing, thiz->outgoing_max*sizeof(struct edge));
	}

	thiz->outgoing[thiz->outgoing_degree].alpha = alpha;
	thiz->outgoing[thiz->outgoing_degree++].next = next;
}


/******************************************************************************
FUNCTION: node_assign_id
DESCRIPTION: 
	every node has a unique ID only for debugging purpose. it is not
	necessary in normal execution.
******************************************************************************/
#ifdef DEBUG_DISPLAY_AC
void node_assign_id (NODE * thiz)
{
	static int unique_id = 100;
	thiz->id = unique_id ++;
}
#endif

/******************************************************************************
FUNCTION: node_edge_compare
DESCRIPTION: Comparison function for qsort. see man qsort.
******************************************************************************/
int node_edge_compare (const void * l, const void * r)
{
	/* 
	According to man page:
	The comparison function must return an integer less than, equal to, or 
	greater than zero if the first argument is considered to be respectively 
	less than,  equal  to,  or greater than the second.  If two members 
	compare as equal, their order in the sorted array is undefined.
	NOTE: Because edge alphas are unique in every node we ignore equivalence
	case for sake of performance.
	*/
	if ( ((struct edge *)l)->alpha >= ((struct edge *)r)->alpha )
		return 1;
	else
		return -1;
}


/******************************************************************************
FUNCTION: node_sort_edges
DESCRIPTION: sort edges alpha.
******************************************************************************/
void node_sort_edges (NODE * thiz)
{
	qsort ((void *)thiz->outgoing, thiz->outgoing_degree, sizeof(struct edge),
			node_edge_compare);
}



short verbosity = 0;

STRING read_file_to_search (const char *filename);
STRING** read_patterns (const char *filename, unsigned int *no_of_patterns);
void print_usage (const char *exec_file);
int match_handler(MATCH * m, int automata_num, int thread_num);

int main(int argc, char **argv)
{
	clock_t start = clock(), end, difference;
	AC_AUTOMATA *aca;
	STRING **patterns, input_buffer;
	unsigned int i, j, no_of_patterns = 6, chunk;
	int clopt;

	/* Command line config*/
	const char *pattern_file = "/content/patterns.txt";
	const char *input_file = "/content/data.txt";
	short timeit = 0;

  verbosity = 1;
  timeit = 1;

	if (verbosity)
		printf("Loading patterns from file - %s\n", pattern_file);
	
	patterns =  read_patterns (pattern_file, &no_of_patterns);

	aca = (AC_AUTOMATA *) malloc(sizeof(AC_AUTOMATA)*NO_OF_THREADS);

	if (verbosity)
		printf("Initialising automata\n");
	
	#pragma omp parallel for shared(aca)
	for(i = 0; i < NO_OF_THREADS; i++)
		ac_automata_init (&aca[i], match_handler);

	if (verbosity)
		printf("Adding strings\n");

	#pragma omp parallel for shared(aca)
	for (i = 0; i < NO_OF_THREADS; i++)
		for (j = 0; j < no_of_patterns/NO_OF_THREADS; j++)
			ac_automata_add_string(&aca[i], &patterns[i][j]);

	input_buffer = read_file_to_search(input_file);

	if (verbosity)
		printf("Locating failure nodes\n");

	#pragma omp parallel for shared(aca)
	for (i = 0; i < NO_OF_THREADS; i++)
			ac_automata_locate_failure (&aca[i]);

	if (verbosity)
		printf("Searching\n");

	#pragma omp parallel for shared(aca)
	for (i = 0; i < NO_OF_THREADS; i++) {
		int id = omp_get_thread_num();
		if (verbosity) {
			printf("In thread: %d, Automata: %d\n", id, i);
		}

		ac_automata_search(&aca[i], &input_buffer, i, id);
	}
	
	if (verbosity)
		printf("Freeing resources\n");

	for (i = 0; i < NO_OF_THREADS; i++)
		ac_automata_release (&aca[i]);

	if (timeit) {
		int msec;
		end = clock();
		difference = end - start;
		msec = difference * 1000 / CLOCKS_PER_SEC;
		printf("\nTotal time taken - %d milliseconds\n", msec);
	}
	
	return 0;
}


STRING read_file_to_search (const char *filename)
{
	STRING buffer;
	FILE *fp;

	fp = fopen(filename, "r");

    fseek(fp, 0, SEEK_END);
    buffer.length = ftell(fp) + 1;
    rewind(fp);

    buffer.str = (ALPHA *) malloc((buffer.length) * sizeof(ALPHA));
    fread(buffer.str, buffer.length - 1, 1, fp);
    buffer.str[buffer.length - 1] = '\0';

    fclose(fp);

    return buffer;
}


STRING** read_patterns (const char *filename, unsigned int *no_of_patterns)
{
	unsigned int i, j, chunk;
	ALPHA *buffer = (ALPHA *) malloc(MAX_PATTERN_SIZE * sizeof(char));
	STRING **patterns = (STRING **) malloc(NO_OF_THREADS * sizeof(STRING *));
	FILE *fp;

	fp = fopen(filename, "r");

	fscanf(fp, "%u\n", no_of_patterns);

	chunk = *no_of_patterns / NO_OF_THREADS;
	for(i = 0; i < NO_OF_THREADS; i++) {
		
		patterns[i] = (STRING *) malloc(chunk * sizeof(STRING));
		
		for(j = 0; j < chunk; j++) {
			if (fscanf(fp, "%s\n", buffer) == -1)
				break;
			
			patterns[i][j].str = (ALPHA *) malloc((strlen(buffer)+1) * sizeof(ALPHA));
			
      printf("%s\n", buffer);
			strcpy(patterns[i][j].str, buffer);
			patterns[i][j].length = strlen(buffer);
			patterns[i][j].id = i + 1;
		}
	}

	fclose(fp);

	return patterns;
}


void print_usage (const char *exec_file)
{
    printf("Usage: %s [-vt] -P pattern_file file1\n", exec_file);
}


int match_handler(MATCH * m, int automata_num, int thread_num)
{
	if (verbosity) {
		unsigned int j;

		// printf ("@ Thread %ld Automata %ld position %ld string(s) ", thread_num, automata_num, m->position);

		// for (j=0; j < m->match_num; j++)
			// printf("%ld (%s), ", m->matched_strings[j].id, m->matched_strings[j].str);
		
		// printf("matched\n");
	}

	return 0;
}

Overwriting ac_omp.c


In [84]:
!gcc ac_omp.c -o ac_omp -fopenmp

In [85]:
!./ac_omp

Loading patterns from file - /content/patterns.txt
ATC
GTG
GTC
ATG
CAA
ATT
Initialising automata
Adding strings
Locating failure nodes
Searching
In thread: 0, Automata: 0
In thread: 1, Automata: 1
Freeing resources

Total time taken - 90 milliseconds
